# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
mydb = mongo.get_database('establishments.json')

database_list = mongo.list_database_names()

if 'establishments.json' in database_list:
    print('The database "establishments.json" exists.')
else:
    print('The database "establishments.json" does not exist.')

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collection_list = db.list_collection_names()

print('Collections in the "uk_food" database:')
for collection_name in collection_list:
    print(collection_name)

In [ ]:
# review the collections in our new database
my_collection = db['uk_food']

In [ ]:
# review a document in the establishments collection
establishments_collection = db['establishments']

document = establishments_collection.find_one()

if document:
    print('Document from the "establishments" collection:')
    print(document)
else:
    print('No document found in the "establishments" collection.')

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    'name': 'Penang Flavours',
    'location': 'Greenwich',
    'cuisine': 'Halal',
    'rating': None
}

In [ ]:
# Insert the new restaurant into the collection
insert_results = establishments_collection.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("Penang Flavours has been added to the 'establishments' collection.")
else:
    print("Failed to add Penang Flavours to the 'establishments' collection.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

projection = {'_id': 0, 'BusinessTypeID': 1, 'BusinessType': 1}

result = establishments_collection.find_one(query, projection)

if result:
    print("BusinessTypeID:", result["BusinessTypeID"])
    print("BusinessType:", result["BusinessType"])
else:
    print("No matching document found.")

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {'name': 'Penang Flavours'}

update_query = {'$set': {'BusinessTypeID': your_business_type_id}}

update_result = establishments_collection.update_one(filter_query, update_query)

In [ ]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("The 'Penang Flavours' restaurant has been updated with BusinessTypeID.")
else:
    print("No document was updated.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

count = establishments_collection.count_documents(query)

print(f'Number of documents with LocalAuthorityName "Dover": {count}')

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many(query)

print(f'Number of documents deleted: {delete_result.deleted_count}')

In [ ]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}

count = establishments_collection.count_documents(query)

if count > 0:
    print('There are remaining documents with LocalAuthorityName "Dover".')
else:
    print('No documents with LocalAuthorityName "Dover" are remaining.')

In [ ]:
# Check that other documents remain with 'find_one'
query = {}

document = establishments_collection.find_one(query)

if document:
    print("One of the remaining documents:")
    print(document)
else:
    print("No remaining documents found.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
update_query = [
    {
        "$set": {
            "longitude": {"$toDouble": "$longitude"},
            "latitude": {"$toDouble": "$latitude"}
        }
    }
]

update_result = establishments_collection.bulk_write(update_query)

print(f"Number of documents updated: {update_result.modified_count}")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_query = [
    {
        "$set": {
            "RatingValue": {"$toInt": "$RatingValue"}
        }
    }
]

update_result = establishments_collection.bulk_write(update_query)

print(f"Number of documents updated: {update_result.modified_count}")

In [ ]:
# Check that the coordinates and rating value are now numbers
query = {}

document = establishments_collection.find_one(query)

if document:
    longitude_type = type(document.get("longitude"))
    latitude_type = type(document.get("latitude"))
    rating_value_type = type(document.get("RatingValue"))

    print("Data types:")
    print(f"Longitude: {longitude_type}")
    print(f"Latitude: {latitude_type}")
    print(f"RatingValue: {rating_value_type}")
else:
    print("No remaining documents found.")